## Fine-tuning of WizardLM-13B on Convfinqa

Install and Load Required Libraries

In [1]:
! pip3 install -q -U transformers
! pip install -q -U datasets
! pip3 install -q -U peft
! pip install -q -U trl
! pip3 install -q -U auto-gptq
! pip3 install -q -U optimum
! pip3 install -q -U bitsandbytes

In [2]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/kmb85/rds/hpc-work/huggingface'

In [3]:
import transformers
import torch
from datasets import load_dataset
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from trl import SFTTrainer

/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


### Load WizardLM-13B and Tokenizer

In [4]:
model_name_or_path = "WizardLM/WizardLM-13B-V1.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4b_quant_type='nf4',
    torch_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    use_safetensors=True,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
    token=""
)

/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kmb85/rds/hpc-work/m

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True,
                                          token="")
tokenizer.pad_token=tokenizer.eos_token

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

### Load LoRA Adapter

In [7]:
config = LoraConfig(
    r=32,
    lora_alpha=16,
    bias="none",
    task_type="CASUAL_LM",
    target_modules=["q_proj", "v_proj"]
)

In [8]:
model=get_peft_model(model, config)

### Dataset preparation

In [9]:
dataset = load_dataset('ChanceFocus/flare-convfinqa', keep_in_memory=True)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'query', 'answer', 'turn', 'dialogue_id'],
        num_rows: 8891
    })
    test: Dataset({
        features: ['id', 'query', 'answer', 'turn', 'dialogue_id'],
        num_rows: 1490
    })
    valid: Dataset({
        features: ['id', 'query', 'answer', 'turn', 'dialogue_id'],
        num_rows: 2213
    })
})

In [11]:
import re

def remove_conversations(text):
    cleaned_text = re.sub(r'Conversations:.*?Question:', 'Question:', text, flags=re.DOTALL)
    return cleaned_text

def add_marker_before_first_occurrence(text, search_string):
    pattern = re.escape(search_string)
    marked_text = re.sub(pattern, '\n###' + search_string, text, count=1)
    return marked_text

In [12]:
def generate_train_prompt(data_point):
    query = data_point['query']
    query = add_marker_before_first_occurrence(query, 'Context:')
    query = add_marker_before_first_occurrence(query, 'Conversation:')
    query = add_marker_before_first_occurrence(query, 'Question:')
    query = add_marker_before_first_occurrence(query, 'Answer:')
    answer = data_point['answer']
    text = f'{query}\n{answer}'
    return {'text': text, 'labels': answer}

In [13]:
train_dataset = dataset['train'].shuffle().map(generate_train_prompt)

Map:   0%|          | 0/8891 [00:00<?, ? examples/s]

In [14]:
validation_dataset = dataset['valid'].shuffle().map(generate_train_prompt)

Map:   0%|          | 0/2213 [00:00<?, ? examples/s]

In [15]:
def generate_test_prompt(data_point):
    query = data_point['query']
    query = add_marker_before_first_occurrence(query, 'Context:')
    query = add_marker_before_first_occurrence(query, 'Question:')
    query = add_marker_before_first_occurrence(query, 'Answer:')
    text = f'{query}\n\n'
    return {'text':text}

In [16]:
test_dataset = dataset['test'].shuffle().map(generate_test_prompt)

Map:   0%|          | 0/1490 [00:00<?, ? examples/s]

### Training

In [17]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=12,
    gradient_accumulation_steps=12,
    learning_rate=0.00003,
    bf16=True,
    num_train_epochs=5,
    save_strategy="epoch",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    output_dir='./experiments',
    remove_unused_columns=False,
    warmup_ratio=0.03,
    logging_strategy='steps',
    evaluation_strategy='steps',
    logging_steps=15,
    label_names=['labels'],
    eval_steps=15,
    group_by_length=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    args=training_args,
    tokenizer=tokenizer,
    dataset_text_field='text',
    peft_config=config,
    max_seq_length=4096
)

Map:   0%|          | 0/8891 [00:00<?, ? examples/s]

Map:   0%|          | 0/2213 [00:00<?, ? examples/s]

/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
model.config.use_cache = False
trainer.state.log_history = True
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kmb85 (cam_kiril). Use `wandb login --relogin` to force relogin


/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.42 GiB. GPU 0 has a total capacity of 79.15 GiB of which 90.38 MiB is free. Including non-PyTorch memory, this process has 79.05 GiB memory in use. Of the allocated memory 66.94 GiB is allocated by PyTorch, and 11.61 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Save the fine-tuned model

In [ ]:
model.save_pretrained(f'WizardLM-13b-hf-convfinqa_batch_size_12_epochs_5')

### Evaluate the fine-tuned model

In [ ]:
total_correct = 0

In [ ]:
test_dataset = test_dataset.shuffle(seed=42)

In [ ]:
import requests
import ast

request = {
    'max_new_tokens': 200,
    'temperature': 0.1,
    'repetition_penalty': 1,
    'top_p': 0.7,
}

url = "http://127.0.0.1:5070/api/v1/generate"
headers = {'Content-Type': 'application/json'}

In [ ]:
def trim_float_string(float_string):
    if float_string.endswith('.0'):
        return float_string[:-2]
    else:
        return float_string

In [ ]:
def extract_and_compare(numerical_output, llm_output):
    numerical_value = float(numerical_output)

    numbers_in_text = re.findall(r'\b\d+\b', llm_output)
    words_in_text = re.findall(r'\b[a-zA-Z]+\b', llm_output)

    llm_values = [float(num) for num in numbers_in_text]

    for word in words_in_text:
        try:
            llm_values.append(float(w2n.word_to_num(word)))
        except ValueError:
            continue

    return any(numerical_value == llm_val for llm_val in llm_values)

In [ ]:
from word2number import w2n

for i in range(len(test_dataset)):
    request['prompt'] = test_dataset[i]['text']
    response = requests.post(url, json=request)

    prediction = ast.literal_eval(response.text)["results"][0]['text'].lower()
    correct_ans = trim_float_string(test_dataset[i]['answer'].lower())

    if correct_ans in prediction:
        total_correct += 1
    else:
        try:
            if extract_and_compare(correct_ans, prediction):
                total_correct += 1
        except:
            continue

In [ ]:
correct_percentage = (total_correct / len(test_dataset)) * 100
print(f'Exact Match(EM) Accuracy {correct_percentage}%')

### Evaluate the base model

In [30]:
test_dataset = test_dataset.shuffle(seed=42)

In [31]:
import requests
import ast

request = {
    'max_new_tokens': 200,
    'temperature': 0.1,
    'repetition_penalty': 1,
    'top_p': 0.7,
}

url = "http://127.0.0.1:5000/api/v1/generate"
headers = {'Content-Type': 'application/json'}

In [32]:
def trim_float_string(float_string):
    if float_string.endswith('.0'):
        return float_string[:-2]
    else:
        return float_string

In [33]:
def extract_and_compare(numerical_output, llm_output):
    numerical_value = float(numerical_output)

    numbers_in_text = re.findall(r'\b\d+\b', llm_output)
    words_in_text = re.findall(r'\b[a-zA-Z]+\b', llm_output)

    llm_values = [float(num) for num in numbers_in_text]

    for word in words_in_text:
        try:
            llm_values.append(float(w2n.word_to_num(word)))
        except ValueError:
            continue

    return any(numerical_value == llm_val for llm_val in llm_values)

In [34]:
total_correct = 0

In [ ]:
from word2number import w2n

for i in range(len(test_dataset)):
    request['prompt'] = test_dataset[i]['text']
    response = requests.post(url, json=request)

    prediction = ast.literal_eval(response.text)["results"][0]['text'].lower()
    correct_ans = trim_float_string(test_dataset[i]['answer'].lower())

    if correct_ans in prediction:
        total_correct += 1
    else:
        try:
            if extract_and_compare(correct_ans, prediction):
                total_correct += 1
        except:
            continue

In [ ]:
correct_percentage = (total_correct / len(test_dataset)) * 100
print(f'Exact Match(EM) Accuracy {correct_percentage}%')